In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, broadcast

spark = SparkSession.builder \
    .appName("RetailPipeline") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.sql.adaptive.enabled", "false") \
    .getOrCreate()




26/02/23 11:13:26 WARN Utils: Your hostname, ASHUTOSHs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.100 instead (on interface en0)
26/02/23 11:13:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/23 11:13:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/23 11:13:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sales_df = spark.read.csv("sales.csv", header=True, inferSchema=True)
customers_df = spark.read.csv("customers.csv", header=True, inferSchema=True)


print("Initial sales partitions:", sales_df.rdd.getNumPartitions())

Initial sales partitions: 1


In [3]:
filtered_sales = sales_df.filter(col("amount") > 15000)
filtered_sales.show()

+--------+-----------+-------+------+----------+
|order_id|customer_id|product|amount|      date|
+--------+-----------+-------+------+----------+
|       1|        101| Mobile| 20000|2024-01-01|
|       2|        102| Laptop| 50000|2024-01-01|
|       4|        103| Mobile| 18000|2024-01-02|
+--------+-----------+-------+------+----------+



In [4]:
sales_agg = filtered_sales.groupBy("customer_id") \
    .agg(sum("amount").alias("total_spent"))
    
print("Partitions after groupBy:", sales_agg.rdd.getNumPartitions())

Partitions after groupBy: 8


In [5]:
final_df = sales_agg.join(
    broadcast(customers_df),
    "customer_id",
    "inner"
)
final_df.show()

+-----------+-----------+------+
|customer_id|total_spent|  city|
+-----------+-----------+------+
|        102|      50000| Delhi|
|        101|      20000|Mumbai|
|        103|      18000|  Pune|
+-----------+-----------+------+



In [6]:
final_df = final_df.repartition(4)
print("Partitions final:", final_df.rdd.getNumPartitions())

Partitions final: 4


In [7]:
final_df.write \
    .mode("overwrite") \
    .partitionBy("city") \
    .parquet("output_data")

print("Final partitions:", final_df.rdd.getNumPartitions())

Final partitions: 4
